In [1]:
import flask
from flask import Flask, request, Response
import tensorflow as tf
import numpy as np
import jsonpickle
import cv2
import json

In [2]:
gender_model = tf.keras.models.load_model("gender_model")
age_model = tf.keras.models.load_model("age_model")

In [47]:
app = Flask(__name__)
@app.route('/api/health',methods = ['GET'])
def health_check():
    #check if models can be loaded correctly
    try:
        gender_model = tf.keras.models.load_model("gender_model")
        age_model = tf.keras.models.load_model("age_model")
        if(gender_model and age_model):
            status = 200
        else:
            status = 404
    except:
        status = 404
        
    return flask.Response(response="\n", status=status, mimetype="application/json")

@app.route('/api/predict',methods = ['POST'])
def predict():
    
    #process data to be predicted
    image_array = np.fromstring(request.data, np.uint8)
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    img = img.astype('float16')
    img = img/255.
    img = np.array([img])
    
    #get gender prediction
    gender_prediction = gender_model.predict(img)
    gender_prediction = np.argmax(gender_prediction)
    
    if(gender_prediction == 0):
        gender_prediction == "Male"
    else:
        gender_prediction == "Female"
        
    #get age prediction
    age_prediction = age_model.predict(img)
    age_prediction = age_prediction[0][0]
    age_prediction = round(age_prediction)
    
    prediction = {"Gender":str(gender_prediction), "Age":str(age_prediction)}
    
    return flask.Response(response = json.dumps(prediction),status = 200, mimetype = 'application/json')

In [ ]:
if __name__ == '__main__':
    app.run(host = '0.0.0.0', port=105)

In [ ]:
#example function to call api via image path
def call_api(image):
    img = open(image,'rb').read()
    response = requests.post('http://localhost:105/api/predict',data = img)
    return json.loads(response.content.decode('utf-8'))